In [10]:
import openpyxl
import json
import os

def extract_tables_to_json(excel_file, output_folder):
    # 엑셀 파일 로드
    wb = openpyxl.load_workbook(excel_file, data_only=True)
    
    # 결과 저장 폴더 생성 (없는 경우)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 모든 시트에 대해 반복
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        sheet_data = {}  # 현재 시트의 모든 테이블 데이터를 저장할 딕셔너리

        # 시트 내의 모든 테이블 확인
        for table in ws.tables.values():
            # 테이블의 데이터 추출
            data = []
            for row in ws[table.ref]:
                # None 값을 제외하고 데이터 추출
                cleaned_row = [cell.value if cell.value is not None else "" for cell in row]
                data.append(cleaned_row)

            if not data or len(data) < 2:
                continue  # 데이터가 없거나 헤더만 있는 경우 다음 테이블로

            # 첫 번째 행을 헤더로 사용하여 데이터 준비
            headers = data[0]
            # 빈 열 제거
            columns_to_keep = [i for i, header in enumerate(headers) if any(row[i] != "" for row in data[1:])]
            filtered_data = [[row[i] for i in columns_to_keep] for row in data]

            # 테이블 데이터 객체 배열 생성
            list_of_data_objects = []
            for row in filtered_data[1:]:
                metadata = dict(zip(headers, row))
                text_summary = " ".join(str(item) for item in row if item != "")
                data_object = {
                    "metadata": metadata,
                    "text": text_summary
                }
                list_of_data_objects.append(data_object)

            # 테이블 이름을 키로 사용하여 데이터 저장
            sheet_data[table.name] = list_of_data_objects

        # JSON 파일로 저장
        if sheet_data:
            json_file_path = os.path.join(output_folder, f"{sheet}.json")
            with open(json_file_path, 'w', encoding='utf-8') as json_file:
                json.dump(sheet_data, json_file, ensure_ascii=False, indent=4)

            print(f"All tables from sheet '{sheet}' have been extracted and saved to '{json_file_path}'")

# 사용 예시
extract_tables_to_json('example.xlsx', 'output_tables')


All tables from sheet '(1) 업무기준' have been extracted and saved to 'output_tables\(1) 업무기준.json'
